In [1]:
import numpy as np
from PlotSettings import MainColor1, plt
from CrossSections import Transfer_Sigma_Low_Energy, Transfer_Sigmav_Low_Energy, sigv_T

## Fits con datos de Yu

In [2]:

#for the plots
DwarfData = np.loadtxt('Data-Sets/DwarfDataPlot.csv', delimiter='\t')
Dwarf_x =     DwarfData[:,0]
Dwarf_y =     DwarfData[:,1]
Dwarf_Err_l = DwarfData[:,2]
Dwarf_Err_r = DwarfData[:,3]
Dwarf_Err_d = DwarfData[:,4]
Dwarf_Err_u = DwarfData[:,5]

GalaxyData = np.loadtxt('Data-Sets/GalaxyDataPlot.csv', delimiter='\t')
Galaxy_x = GalaxyData[:,0]
Galaxy_y = GalaxyData[:,1]
Galaxy_Err_l = GalaxyData[:,2]
Galaxy_Err_r = GalaxyData[:,3]
Galaxy_Err_d = GalaxyData[:,4]
Galaxy_Err_u = GalaxyData[:,5]

ClusterData = np.loadtxt('Data-Sets/ClusterDataPlot.csv', delimiter='\t')
Cluster_x = ClusterData[:,0]
Cluster_y = ClusterData[:,1]
Cluster_Err_l = ClusterData[:,2]
Cluster_Err_r = ClusterData[:,3]
Cluster_Err_d = ClusterData[:,4]
Cluster_Err_u = ClusterData[:,5]

# For the fit
OrderedData = np.loadtxt('Data-Sets/Datos_ordenados.csv')
x_data = OrderedData[:,0]
y_data = OrderedData[:,1]
err_x_data = OrderedData[:,2]
err_y_data = OrderedData[:,3]


In [28]:
from scipy.optimize import minimize

alpha_EM = 1/137
e = np.sqrt(4*np.pi*alpha_EM)


def compute_chi2(free_params, x=x_data, data=y_data, err=(err_x_data, err_y_data)):
    
    M, m = free_params
    errx, erry = err
    model = [Transfer_Sigmav_Low_Energy(x, g=e, M=M, m=m) for x in x]

    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2x + chi2y

priors = [0.01, 10]

#Sol = minimize(compute_chi2, priors,  method='Nelder-Mead', tol=1e-6)

print(compute_chi2((0.02, 2)) )
print(compute_chi2((0.03, 15)) )



2250574.547908529
2250574.5479085315


In [29]:
10**(0.3)

1.9952623149688795

## Fits con Datos de Camila

In [54]:
#Import Data from Camila Correa
CamilaData_Fig6 = np.loadtxt('Data-Sets/Data_Fig6_Correa_2021.txt')

VelocityData = CamilaData_Fig6[:,0]
Velocity_16_Percentile = CamilaData_Fig6[:,1]
Velocity_84_Percentile = CamilaData_Fig6[:,2]
VelocityData_Err = ( (VelocityData - Velocity_16_Percentile) + (Velocity_84_Percentile - VelocityData) ) /2

CrossSectionData = CamilaData_Fig6[:,3]
CrossSectionData_16_Percentile = CamilaData_Fig6[:,4]
CrossSectionData_84_Percentile = CamilaData_Fig6[:,5]

CrossSectionData_Err = ((CrossSectionData - CrossSectionData_16_Percentile) + (CrossSectionData_84_Percentile - CrossSectionData) )/2


In [55]:
def CamilaTranser(v, M, m):
    alpha = 0.01
    w = 300*(M/10)*(10/m)
    sigma0T = 274.4*(alpha/0.01)**2*(m/10)*(10/M)**4
    return sigma0T*4*w**4/v**4 * (2*np.log(1 + v**2/(2*w**2)) - np.log(1 + v**2/w**2) )


# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [CamilaTranser(x, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y #+ chi2x


Fits for Figure 6

In [57]:
from scipy.optimize import minimize

M_ini, m_ini = 1, 10
initial = [M_ini, m_ini]

soln = minimize(compute_chi2, initial, method='Nelder-Mead')

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))


Best Fit Parameters
M = 26.90 MeV
m = 112.78 GeV
Chi-squared: 22.40065400937851


Fits for Figure 7:

In [56]:
CamilaData_Fig7 = np.loadtxt('Data-Sets/Data_Fig7_Correa_2021.txt')
VelocityData = CamilaData_Fig7[:,0]
Velocity_16_Percentile = CamilaData_Fig7[:,1]
Velocity_84_Percentile = CamilaData_Fig7[:,2]
VelocityData_Err = ( (VelocityData - Velocity_16_Percentile) + (Velocity_84_Percentile - VelocityData) ) /2

CrossSectionData = CamilaData_Fig7[:,3]
CrossSectionData_16_Percentile = CamilaData_Fig7[:,4]
CrossSectionData_84_Percentile = CamilaData_Fig7[:,5]

CrossSectionData_Err = ((CrossSectionData - CrossSectionData_16_Percentile) + (CrossSectionData_84_Percentile - CrossSectionData) )/2


In [58]:

def Integrand_Camila_SigmaV(v, v0, M, m):
    return CamilaTranser(v, M, m)*v*np.exp(-0.5*v**2/v0**2)*v**2

from scipy.integrate import quad

def Camila_SigmaV(v0, M, m):
    sigma2_MB = v0**2*np.pi*(3*np.pi - 8)/np.pi
    vmax = 2*np.sqrt(sigma2_MB)

    Prefactor = 4*np.pi/((2*np.pi*v0**2)**1.5 * m)
    Integral = quad(Integrand_Camila_SigmaV, 0.1, vmax, args=(v0, M, m))[0]
    return Prefactor*Integral

# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [Camila_SigmaV(x, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y #+ chi2x

In [63]:
soln = minimize(compute_chi2, initial, method='BFGS')

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))
print("\n Covariance matrix:" )
soln.hess_inv

Best Fit Parameters
M = -1241.76 MeV
m = 17732.61 GeV
Chi-squared: 268.46610299515066

 Covariance matrix:


array([[ 0.00490113, -0.06983633],
       [-0.06983633,  0.99509887]])